In [12]:
!pip install -U pymilvus
!pip install "pymilvus[model]"

In [2]:
import pandas as pd

In [3]:
# View Config
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 400)

In [4]:
df = pd.read_csv('netflix_titles.csv')

In [5]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."


In [6]:
def create_textual_representation(row):
    textual_representation = f"""Type: {row['type']},
    Title: {row['title']},
    Director: {row['director']},
    Cast: {row['cast']},
    Release Year: {row['release_year']},
    Release Year: {row['listed_in']},
    Description: {row['description']},
    """

    return textual_representation

In [7]:
df['textual_representation'] = df.apply(create_textual_representation, axis=1)

In [8]:
print(df['textual_representation'].values[0])

Type: Movie,
    Title: Dick Johnson Is Dead,
    Director: Kirsten Johnson,
    Cast: nan,
    Release Year: 2020,
    Release Year: Documentaries,
    Description: As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.,
    


In [20]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

In [101]:
docs = [i for i in df['textual_representation']]       

In [129]:
import numpy as np

dim = 768

index = faiss.IndexFlatL2(dim)

X = np.zeros((len(df['textual_representation']), dim), dtype='float32')

In [131]:
X.shape

(8807, 768)

In [149]:
for i, representation in enumerate(df['textual_representation']):
    embedding = embedding_fn.encode_documents([representation])

    X[i] = np.array(embedding)
        
    if i % 30 == 0:
        print('processed', str(i), 'instances')

processed 0 instances
processed 30 instances
processed 60 instances
processed 90 instances
processed 120 instances
processed 150 instances
processed 180 instances
processed 210 instances
processed 240 instances
processed 270 instances
processed 300 instances
processed 330 instances
processed 360 instances
processed 390 instances
processed 420 instances
processed 450 instances
processed 480 instances
processed 510 instances
processed 540 instances
processed 570 instances
processed 600 instances
processed 630 instances
processed 660 instances
processed 690 instances
processed 720 instances
processed 750 instances
processed 780 instances
processed 810 instances
processed 840 instances
processed 870 instances
processed 900 instances
processed 930 instances
processed 960 instances
processed 990 instances
processed 1020 instances
processed 1050 instances
processed 1080 instances
processed 1110 instances
processed 1140 instances
processed 1170 instances
processed 1200 instances
processed 1230

In [151]:
index.add(X)

In [153]:
faiss.write_index(index, 'index') 